# Predicting Antigenic Outliers by Year using an Isolation Forest

Isolation forests are a method of finding outliers in a dataset using data partitioning. The method works by repetitively partitioning a random feature (of an n-dimensional dataset) between its minimum and maximum values until a sample is isolated or all samples in a partition have the same value. Samples that are higher up in the isolation forst tree are considered outliers where samples deeper in the tree data structure are less likely to be outliers. 

You can read more on the [wikipedia](https://en.wikipedia.org/wiki/Isolation_forest) or the documentation by [sci-kit learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)

This document details the process of predicting outliers (antigenically distinct sequences) in a series of years which are 'seen' step-wise manner. 

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
### Helper functions for plotting

# Only get the first of each handle when plotting
def unique_everseen(seq, key=None):
    seen = set()
    seen_add = seen.add
    return [x for x,k in zip(seq,key) if not (k in seen or seen_add(k))]

#  Returns tuple of handles, labels for axis ax, after reordering them to conform to the label order `order`, and if unique is True, after removing entries with duplicate labels.
def reorder_legend(ax=None,order=None,unique=False):
    if ax is None: ax=plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0])) # sort both labels and handles by labels
    if order is not None: # Sort according to a given list (not necessarily complete)
        keys=dict(zip(order,range(len(order))))
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda t,keys=keys: keys.get(t[0],np.inf)))
    if unique:  labels, handles= zip(*unique_everseen(zip(labels,handles), key = labels)) # Keep only the first of each handle
    ax.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.25,1))
    return(handles, labels)

### Helper functions for misc use

# Get unique items in a list
def unique(list):
    unique_items=[]
    for i in list:
        if i not in unique_items:
            unique_items.append(i)
        else:
            pass
    return unique_items

In [ ]:
# Constants used throughout cells

# Colors for plotting by cluster
CLUSTER_COLORS={'HK68':'dodgerblue',
                'EN72':'magenta',
                'VI75':'olive', 
                'TX77':'aqua', 
                'BK79':'red', 
                'SI87':'gold', 
                'BE89':'grey',
                'BE92':'salmon', 
                'WU95':'teal', 
                'SY97':'maroon', 
                'FU02':'palegreen'}
# All features in dataset
FEATURES=['distRoot', 'branch_length', 'hydrophobicity', 'charge', 'boman', 'instability', 'isoelectric_point']
# Features excluding phylogenetic measurements
NO_PHYLO_FEATURES=['hydrophobicity', 'charge', 'boman', 'instability', 'isoelectric_point']

In [ ]:
# Load data
file_path='../../data/processedData/sequence_physioproperties.csv'
physio_data=pd.read_csv(file_path)

# Get numerical features and scale them to zero mean and unit variance
ss=StandardScaler()
to_scale=physio_data[FEATURES]
scaled_df=pd.DataFrame(ss.fit_transform(to_scale), 
                       columns=to_scale.columns)

# Add the categorical variable back in
scaled_df['cluster'] = physio_data['cluster']

year=[x.split('/')[2] for x in physio_data['seq_id'].to_list()]
for i in range(0, len(year)):
    if int(year[i]) > 67:
        year[i]=int('19'+year[i])
    else:
        year[i]=int('20'+year[i])

physio_data = scaled_df
physio_data['year']=year

In [ ]:
# Get the first two clusters of data ('HK68' and 'EN72')
initial_data=physio_data[physio_data['year'] < 1974]
current_clusters=initial_data.groupby('cluster')

# Plot distance from the root and hydrophobicity
fig, ax = plt.subplots()
for name, group in current_clusters:
    ax.scatter(x=group['distRoot'], 
               y=group['hydrophobicity'],
               c=CLUSTER_COLORS.get(name),
               label=name)
ax.legend(loc='upper right')
reorder_legend(ax, CLUSTER_COLORS.keys())
plt.show()

In [ ]:
# Set current_data to first two clusters of sequences
current_data=initial_data.copy()
seen_clusters=unique(initial_data['cluster'].to_list())

#fig, axs = plt.subplots(3, 3, figsize=(12,10))

SHAPE = {1: 'o',
         -1: 'x'}

nophylo_features_results = []

plt_row=0
plt_col=0
    
for y in range(1975, 2004):
    next_data=physio_data[physio_data['year'] == y].copy()
    n_test=next_data.shape[0]
    
    next_clusters=unique(next_data['cluster'].to_list())
    next_clusters=[x for x in next_clusters if x not in seen_clusters]
    if len(next_clusters) == 1:
        mr_cluster=next_clusters[0]
        seen_clusters.append(mr_cluster)
    elif len(next_clusters) > 1:
        print(y, next_clusters)
        break
    else:
        pass

    if n_test == 0:
        continue
    else:
        # Training the model
        clf=IsolationForest(random_state=0)
        clf.fit(current_data[NO_PHYLO_FEATURES])
        
        # Predicting whether the next set of data are outliers
        y_pred=clf.predict(next_data[NO_PHYLO_FEATURES])
        next_clusters=next_data['cluster'].to_list()
        false_positive=0
        true_positive=0
        false_negative=0
        true_negative=0
        for i in range(0, len(y_pred)):
            if next_clusters[i] == mr_cluster and y_pred[i] == -1:
                true_positive += 1
            elif next_clusters[i] == mr_cluster and y_pred[i] == 1:
                false_negative += 1
            elif next_clusters[i] != mr_cluster and y_pred[i] == -1:
                false_positive += 1
            else:
                true_negative += 1
        
        print(f'{mr_cluster} ({y}): Accuracy: {(true_positive + true_negative)/(true_positive + true_negative + false_positive + false_negative):4f}')
        # (TP + TN)/(TP + TN + FP + FN)
        
        print(next_clusters)
        print(y_pred, '\n')
        
        current_data=current_data.append(next_data)